In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime
import time
import os

In [ ]:
# Create a list that contains the URL for each page.
pages = []

# There are 130 pages in the entire set.
for page_number in range(1, 130):
    url_start = 'https://www.centralcharts.com/en/price-list-ranking/'
    url_end = 'ALL/asc/ts_19-us-nasdaq-stocks--qc_1-alphabetical-order?p='
    url = url_start + url_end + str(page_number)
    pages.append(url)

# Create a list containing the <th> tag contents.
webpage = requests.get(pages[0])
soup = bs(webpage.text, 'html.parser')
stock_table = soup.find('table', class_='tabMini tabQuotes')
th_tag_list = stock_table.find_all('th')

# Take the <th> list and remove the extra attributes to
# get just the text portion of each tag. Use this text for the 
# column header labels in the dataframe.
headers = []
for each_tag in th_tag_list:
    title = each_tag.text
    headers.append(title)

headers[0] = 'Name'

new_headers = []
for header in headers:
    if header not in ('Cap.', 'Issued Cap.', ''):
        new_headers.append(header)
headers = new_headers
stock_df = pd.DataFrame(columns = headers)


# Cycle through each page. 
for page in pages:
    webpage = requests.get(page)
    soup = bs(webpage.text, 'html.parser')

    # Check to see if the page contains a table. If it does,
    # create a list of <tr> tags. If not, go to the next page.
    if soup.find('table'):
        stock_table = soup.find('table', class_='tabMini tabQuotes')
        tr_tag_list = stock_table.find_all('tr')

        # Cycle through the <tr> list. For each
        # row, find the <td> tags within the row. Then
        # obtain the text within each <td> tag. Lastly, 
        # place the text in the last row of the dataframe.
        for each_tr_tag in tr_tag_list[1:]:
            td_tag_list = each_tr_tag.find_all('td')

            row_values = []
            for each_td_tag in td_tag_list[0:7]:
                new_value = each_td_tag.text.strip()
                row_values.append(new_value)

            stock_df.loc[len(stock_df)] = row_values


# Change the data types & sort by trading volume
stock_df[['Name', 'Current price', 'Change(%)', 'Open','High', 'Low']] = \
    stock_df[['Name', 'Current price', 'Change(%)', 'Open', 'High', 'Low']] \
    .astype(str)

stock_df.replace({'Current price': {',':''},
                  'Change(%)': {',':'', '%':''},
                  'Open': {',':''},
                  'High': {',':''},
                  'Low': {',':''},
                  'Volume': {',':''}
}, regex=True, inplace=True)


stock_df['timestamp'] = datetime.now()

# stock_df.to_csv()
nasdaq_df = stock_df.rename(columns={'Current price': 'Current_price', 'Change(%)': 'Change'})
# if not os.path.exists(data_path):
#     os.makedirs(data_path)

csv_time = str(datetime.now())
csv_time = csv_time.replace(" ","")
csv_time = csv_time.replace(":","")
csv_time = csv_time.replace("-","")

csv_time = csv_time[0:14]
csv_name = 'nasdaq_'+csv_time+'.csv'
data_path = '/lakehouse/default/Files/'+csv_name
nasdaq_df.to_csv(data_path, index=False)

In [ ]:
csv_read_path = "abfss://Hacathon@onelake.dfs.fabric.microsoft.com/Stock_data_lake.Lakehouse/Files/"+csv_name
df = spark.read.format("csv").option("header", True).load(csv_read_path)

df = df.withColumn("timestamp",df.timestamp.cast("Timestamp"))
df = df.withColumn("Current_price",df.Current_price.cast("Double"))
df = df.withColumn("Change",df.Change.cast("Double"))
df = df.withColumn("Open",df.Open.cast("Double"))
df = df.withColumn("High",df.High.cast("Double"))
df = df.withColumn("Low",df.Low.cast("Double"))
df = df.withColumn("Volume",df.Volume.cast("Integer"))

delta_file_path = "abfss://Hacathon@onelake.dfs.fabric.microsoft.com/Stock_data_lake.Lakehouse/Tables/nasdaq"
df.write.mode("append").format("delta").save(delta_file_path)

In [ ]:
# Create a list that contains the URL for each page.
pages = []

# There are 94 pages in the entire set.
for page_number in range(1, 94):
    url_start = 'https://www.centralcharts.com/en/price-list-ranking/'
    url_end = 'ALL/asc/ts_29-us-nyse-stocks--qc_1-alphabetical-order?p='
    url = url_start + url_end + str(page_number)
    pages.append(url)


# Create a list containing the <th> tag contents.
webpage = requests.get(pages[0])
soup = bs(webpage.text, 'html.parser')
stock_table = soup.find('table', class_='tabMini tabQuotes')
th_tag_list = stock_table.find_all('th')

# Take the <th> list and remove the extra attributes to
# get just the text portion of each tag. Use this text for the 
# column header labels in the dataframe.
headers = []
for each_tag in th_tag_list:
    title = each_tag.text
    headers.append(title)

headers[0] = 'Name'

new_headers = []
for header in headers:
    if header not in ('Cap.', 'Issued Cap.', ''):
        new_headers.append(header)
headers = new_headers
stock_df = pd.DataFrame(columns = headers)


# Cycle through each page. 
for page in pages:
    webpage = requests.get(page)
    soup = bs(webpage.text, 'html.parser')

    # Check to see if the page contains a table. If it does,
    # create a list of <tr> tags. If not, go to the next page.
    if soup.find('table'):
        stock_table = soup.find('table', class_='tabMini tabQuotes')
        tr_tag_list = stock_table.find_all('tr')

        # Cycle through the <tr> list. For each
        # row, find the <td> tags within the row. Then
        # obtain the text within each <td> tag. Lastly, 
        # place the text in the last row of the dataframe.
        for each_tr_tag in tr_tag_list[1:]:
            td_tag_list = each_tr_tag.find_all('td')

            row_values = []
            for each_td_tag in td_tag_list[0:7]:
                new_value = each_td_tag.text.strip()
                row_values.append(new_value)

            stock_df.loc[len(stock_df)] = row_values


# Change the data types & sort by trading volume
stock_df[['Name', 'Current price', 'Change(%)', 'Open','High', 'Low']] = \
    stock_df[['Name', 'Current price', 'Change(%)', 'Open', 'High', 'Low']] \
    .astype(str)

stock_df.replace({'Current price': {',':''},
                  'Change(%)': {',':'', '%':''},
                  'Open': {',':''},
                  'High': {',':''},
                  'Low': {',':''},
                  'Volume': {',':''}
}, regex=True, inplace=True)

# stock_df.replace({'Current price': {',':'', '-':'1'},
#                   'Change(%)': {',':'', '-':'1', '%':''},
#                   'Open': {',':'', '-':'1'},
#                   'High': {',':'', '-':'1'},
#                   'Low': {',':'', '-':'1'},
#                   'Volume': {',':'', '-':'1'}
# }, regex=True, inplace=True)

# stock_df[['Current price', 'Change(%)', 'Open', 'High', 'Low', 'Volume']] = \
#     stock_df[['Current price', 'Change(%)', 'Open', 'High', 'Low', 'Volume']]. \
#     apply(pd.to_numeric)


stock_df['timestamp'] = datetime.now()

# stock_df.to_csv()
nyse_df = stock_df.rename(columns={'Current price': 'Current_price', 'Change(%)': 'Change'})
# if not os.path.exists(data_path):
#     os.makedirs(data_path)

csv_time = str(datetime.now())
csv_time = csv_time.replace(" ","")
csv_time = csv_time.replace(":","")
csv_time = csv_time.replace("-","")

csv_time = csv_time[0:14]
csv_name = 'nyse_'+csv_time+'.csv'
data_path = '/lakehouse/default/Files/'+csv_name
nyse_df.to_csv(data_path, index=False)

In [ ]:
csv_read_path = "abfss://Hacathon@onelake.dfs.fabric.microsoft.com/Stock_data_lake.Lakehouse/Files/"+csv_name
df = spark.read.format("csv").option("header", True).load(csv_read_path)

df = df.withColumn("timestamp",df.timestamp.cast("Timestamp"))
df = df.withColumn("Current_price",df.Current_price.cast("Double"))
df = df.withColumn("Change",df.Change.cast("Double"))
df = df.withColumn("Open",df.Open.cast("Double"))
df = df.withColumn("High",df.High.cast("Double"))
df = df.withColumn("Low",df.Low.cast("Double"))
df = df.withColumn("Volume",df.Volume.cast("Integer"))

delta_file_path = "abfss://Hacathon@onelake.dfs.fabric.microsoft.com/Stock_data_lake.Lakehouse/Tables/nyse"
df.write.mode("append").format("delta").save(delta_file_path)